# load and process USDA data

## price data

In [18]:
from pandas import read_csv, read_excel, concat
from json import dump

prices = concat([read_csv("Vegetable-Prices-2022.csv").rename(columns={"Vegetable": "Food"}),
                 read_csv("Fruit-Prices-2022.csv").rename(columns={"Fruit": "Food"})]).set_index("Food")
display(prices)
prices = prices[prices["Form"] != "Canned"]

# 1 Cup = 0.236 L
food_info = {}
for food, row in prices.iterrows():
    food_info[food] = {
        "price": row["RetailPrice"],
        "cupEQ": row["CupEquivalentSize"],
        "unit": row["CupEquivalentUnit"],
        "yield": row["Yield"],
    }

with open("food_info.json", 'w') as jsonOut:
    dump(food_info, jsonOut, indent=3)

,Form,RetailPrice,RetailPriceUnit,Yield,CupEquivalentSize,CupEquivalentUnit,CupEquivalentPrice
Food,,,,,,,
Acorn squash,Fresh,1.2136,per pound,0.4586,0.4519,pounds,1.1961
Artichoke,Fresh,2.4703,per pound,0.3750,0.3858,pounds,2.5415
Artichoke,Canned,3.4498,per pound,0.6500,0.3858,pounds,2.0476
Asparagus,Fresh,2.9531,per pound,0.4938,0.3968,pounds,2.3731
Asparagus,Canned,3.4328,per pound,0.6500,0.3968,pounds,2.0958
...,...,...,...,...,...,...,...
Raspberries,Fresh,7.7338,per pound,0.9600,0.3197,pounds,2.5753
Raspberries,Frozen,6.1590,per pound,1.0000,0.3307,pounds,2.0368
Strawberries,Fresh,2.9682,per pound,0.9400,0.3197,pounds,1.0094


## nutritional data

In [29]:
# create mappings
## nutrient IDs -> nutrient names
# nutrients = read_csv("FoodData_Central_csv_2024-10-31/sub_sample_result.csv").set_index("food_nutrient_id")
# nutrient_names = dict(zip(nutrients.index.to_list(), nutrients["nutrient_name"].to_list()))
nutrients = read_csv("nutrient.csv").set_index("id")
nutrient_names = {}
for ID, row in nutrients.iterrows():
    nutrient_names[ID] = {"name": row["name"], "unit": row["unit_name"]}
display(nutrient_names)
# food IDs -> food names
food_metadata = read_csv("food.csv").set_index("fdc_id")
food_names = dict(zip(food_metadata.index.to_list(), food_metadata["description"].to_list()))

{2047: {'name': 'Energy (Atwater General Factors)', 'unit': 'KCAL'},
 2048: {'name': 'Energy (Atwater Specific Factors)', 'unit': 'KCAL'},
 1001: {'name': 'Solids', 'unit': 'G'},
 1002: {'name': 'Nitrogen', 'unit': 'G'},
 1003: {'name': 'Protein', 'unit': 'G'},
 1004: {'name': 'Total lipid (fat)', 'unit': 'G'},
 1005: {'name': 'Carbohydrate, by difference', 'unit': 'G'},
 1006: {'name': 'Fiber, crude (DO NOT USE - Archived)', 'unit': 'G'},
 1007: {'name': 'Ash', 'unit': 'G'},
 1008: {'name': 'Energy', 'unit': 'KCAL'},
 1009: {'name': 'Starch', 'unit': 'G'},
 1010: {'name': 'Sucrose', 'unit': 'G'},
 1011: {'name': 'Glucose', 'unit': 'G'},
 1012: {'name': 'Fructose', 'unit': 'G'},
 1013: {'name': 'Lactose', 'unit': 'G'},
 1014: {'name': 'Maltose', 'unit': 'G'},
 1015: {'name': 'Amylose', 'unit': 'G'},
 1016: {'name': 'Amylopectin', 'unit': 'G'},
 1017: {'name': 'Pectin', 'unit': 'G'},
 1018: {'name': 'Alcohol, ethyl', 'unit': 'G'},
 1019: {'name': 'Pentosan', 'unit': 'G'},
 1020: {'name'

In [ ]:
# nutrients = read_csv("FoodData_Central_csv_2024-10-31/fndds_ingredient_nutrient_value.csv").set_index("ingredient code")
foods = read_csv("../food_nutrient.csv").set_index("fdc_id")
foods = foods[foods["amount"] > 0]
display(foods)

foods_json = {}
for fdc, row in foods.iterrows():
    if fdc not in foods_json:
        foods_json[fdc] = {
            "name": food_names[fdc],
            "nutrients": {nutrient_names[row["nutrient_id"]]["name"]: row["amount"]}
        }
    else:    foods_json[fdc]["nutrients"].update({nutrient_names[row["nutrient_id"]]["name"]: row["amount"]})

In [26]:
foods

,id,nutrient_id,amount,data_points,derivation_id,min,max,median,loq,footnote,min_year_acquired,percent_daily_value
fdc_id,,,,,,,,,,,,
1105904,13706930,1293,53.33,NaN,71.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1105904,13706916,1008,867.00,NaN,71.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1105904,13706928,1258,13.33,NaN,71.0,NaN,NaN,NaN,NaN,NaN,NaN,10.0
1105904,13706929,1292,20.00,NaN,71.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1105904,13706914,1004,93.33,NaN,71.0,NaN,NaN,NaN,NaN,NaN,NaN,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2721948,34555377,1005,17.86,NaN,71.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0
2721948,34555389,1253,107.00,NaN,71.0,NaN,NaN,NaN,NaN,NaN,NaN,10.0
2721948,34555392,1292,14.29,NaN,71.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
from json import dump
with open("../food_nutrients.json", 'w') as jsonOut:
    dump(foods_json, jsonOut, indent=3)

In [23]:
# only examine fresh (raw) foods, since this is the scope of this project
from json import load, dump
with open("../food_nutrients.json", 'r') as jsonIn:
    foods_json = load(jsonIn)

fresh_foods_json = {}
for ID, content in foods_json.items():
    if " raw" not in str(content["name"]):  continue
    fresh_foods_json[ID] = content

# display(set(x["name"] for x in fresh_foods_json.values()))

with open("fresh_foods_nutrients.json", 'w') as jsonOut:
    dump(fresh_foods_json, jsonOut, indent=3)


fresh_foods_names = {}
for ID, content in fresh_foods_json.items():
    name = content["name"]
    fresh_foods_names[name] = content
    fresh_foods_names[name].pop("name")
    fresh_foods_names[name].update({"id": ID})

with open("fresh_foods_nutrients_names.json", 'w') as jsonOut:
    dump(fresh_foods_names, jsonOut, indent=3)

In [ ]:
with open("food_names.json", 'w') as jsonOut:
    dump(food_names, jsonOut, indent=3)

with open("nutrient_names.json", 'w') as jsonOut:
    dump(nutrient_names, jsonOut, indent=3)

## physiological data

In [13]:
# sourced for an active, 28 year-old, 150lb, man   https://www.nal.usda.gov/human-nutrition-and-food-safety/dri-calculator/results

# macronutritional needs, with noted adjustments
from pandas import read_csv, concat
from math import inf

macro_needs = read_csv("macronutritional_needs.csv").set_index("Macronutrient").drop("Trans fatty acids", axis=0)
macro_needs["low_bound"] = [""]*len(macro_needs)
macro_needs["high_bound"] = [""]*len(macro_needs)
macro_needs["units"] = [""]*len(macro_needs)
for nutrient, row in macro_needs.iterrows():
    if "-" in row["Recommended Intake Per Day"]:
        minimum, maximum = row["Recommended Intake Per Day"].split("-")
        macro_needs.at[nutrient, "low_bound"] = minimum.strip()
        macro_needs.at[nutrient, "high_bound"] = maximum.split()[0]
        macro_needs.at[nutrient, "units"] = maximum.split()[1]
    elif nutrient == "Saturated fatty acids":
        macro_needs.at[nutrient, "high_bound"] = str(round((3000/ 10 /9), 1))  # 10% of total calories
        macro_needs.at[nutrient, "low_bound"] = str(0)
        macro_needs.at[nutrient, "units"] = "grams"
    elif nutrient == "Dietary Cholesterol":
        macro_needs.at[nutrient, "high_bound"] = str(800)  # arbitrary but a reasonable limit
        macro_needs.at[nutrient, "low_bound"] = str(0)
        macro_needs.at[nutrient, "units"] = "milligrams"
    elif nutrient == "Total Water":
        macro_needs.at[nutrient, "low_bound"] = "0.37"
        macro_needs.at[nutrient, "high_bound"] = row["Recommended Intake Per Day"].split()[0]
        macro_needs.at[nutrient, "units"] = row["Recommended Intake Per Day"].split()[1]
    elif nutrient == "Protein":
        macro_needs.at[nutrient, "low_bound"] = "100"
        macro_needs.at[nutrient, "high_bound"] = "150"
        macro_needs.at[nutrient, "units"] = "grams"
    elif "As low" not in row["Recommended Intake Per Day"]:
        macro_needs.at[nutrient, "low_bound"] = row["Recommended Intake Per Day"].split()[0]
        macro_needs.at[nutrient, "high_bound"] = inf
        macro_needs.at[nutrient, "units"] = row["Recommended Intake Per Day"].split()[1]
        
macro_needs.drop("Recommended Intake Per Day", axis=1, inplace=True)

# micronutritional needs
vitamin_needs = read_csv("vitamin_needs.csv").set_index("Vitamin").rename(columns={"Recommended Intake Per Day": "low_bound", "Tolerable UL Intake Per Day": "high_bound"}).replace("ND", inf).fillna("0 milligrams")
vitamin_needs["units"] = [""]*len(vitamin_needs)
for nutrient, row in vitamin_needs.iterrows():
    val, unit = row["low_bound"].split()
    vitamin_needs.at[nutrient, "low_bound"] = val
    vitamin_needs.at[nutrient, "units"] = unit
    vitamin_needs.at[nutrient, "high_bound"] = str(row["high_bound"]).split()[0]

mineral_needs = read_csv("mineral_needs.csv").set_index("Mineral").rename(columns={"Recommended Intake Per Day": "low_bound", "Tolerable UL Intake Per Day": "high_bound"}).replace("ND", inf)
mineral_needs["units"] = [""]*len(mineral_needs)
for nutrient, row in mineral_needs.iterrows():
    val, unit = row["low_bound"].split()
    mineral_needs.at[nutrient, "low_bound"] = val
    mineral_needs.at[nutrient, "units"] = unit
    mineral_needs.at[nutrient, "high_bound"] = str(row["high_bound"]).split()[0]


# combining the nutritional sources
nutrition = concat([macro_needs, vitamin_needs, mineral_needs])
display(nutrition)
print(nutrition.shape)

from json import dump
with open("nutrition.json", 'w') as jsonOut:
    dump(nutrition.T.to_dict(), jsonOut, indent=3)

,low_bound,high_bound,units
Carbohydrate,331,478,grams
Total Fiber,41,inf,grams
Protein,100,150,grams
Fat,65,114,grams
Saturated fatty acids,0,33.3,grams
α-Linolenic Acid,1.6,inf,grams
Linoleic Acid,17,inf,grams
Dietary Cholesterol,0,800,milligrams
Total Water,0.37,3.7,liters
Vitamin A,900,"3,000",mcg


(39, 3)


# define the constraints for each food

# load the previously defined mappings of price, food contents, and nutritional needs

In [14]:
from json import load

with open("food_info.json", 'r') as jsonIn:
    food_info = load(jsonIn)

with open("fresh_foods_nutrients_names.json", 'r') as jsonIn:
    fresh_foods_json = load(jsonIn)

with open("nutrition.json", 'r') as jsonIn:
    nutrition = load(jsonIn)

## finding matches between the data sources:  nutritional need and food composition

In [ ]:
for food, pricing in food_info.items():
    for food2, nutrients in fresh_foods_json.items():
        if 

for nutrient, content in nutrition.items():
    nutrient_foods = {}
    for food, pricing in food_info.items():

        if nutrient not in fresh_foods_json[food]:  continue

In [16]:
# upload my custom model construction API
from modelseedpy.core.optlanghelper import *

# define all of the relevant variables
##NOTE creating integer variables for servings of food, since these are easier to follow than fractional servings
variables = {}
for food, pricing in food_info.items():
    variables[food] = tupVariable(food, Bounds(0, 1000), "integer")
    variables[food+"_bin"] = tupVariable(food+"_bin", Bounds(0, 1), "binary")


constraints = {}
# nutrition constraint
#NOTE eq:  nutrient_lowBound <= sum_n( sum_f( var_f * amount_n,f ) ) <= nutrient_highBound
for nutrient, content in nutrition.items():
    nutrient_foods = {}
    for food, pricing in food_info.items():
        if nutrient not in fresh_foods_json[food]:  continue
        if food not in nutrient_foods:   nutrient_foods[food] = {}
        nutrient_foods[food].update({"elements": [variables[food].name, fresh_foods_json[food][nutrient]], "operation": "Mul"})
    constraints[nutrient] = tupConstraint(name=nutrient, 
                                          bounds=Bounds(nutrition.at[nutrient, "low_bound"], nutrition.at[nutrient, "high_bound"]), 
                                          expr={"elements": [nutrient_foods], "operation": "Add"})

# volume constraint
#NOTE eq:  5 cups <= sum_f(var_f [lb or pint] * cupsPerServing_f [cup/lb or cup/pint]) <= 20 cups
volume_expression = {}
for food, pricing in food_info.items():
    volume_expression.update({"elements": [variables[food].name, pricing["cupEQ"]], "operation": "Mul"})
constraints["volume"] = tupConstraint(name="volume", bounds=Bounds(5, 20), expr=volume_expression)

# number of foods constraint
for food, pricing in food_info.items():
    constraints[food+"_bin"] = tupConstraint(food+"_bin", bounds=Bounds(0,0),   #TODO double-check that this constraint should be limited to zero
                                             expr={"elements": [variables[food].name, -variables[food].ub * variables[food+"_bin"]], "operation": "Mul"})
constraints["foods"] = tupConstraint(name="foods", bounds=Bounds(5, 20), expr={"elements": [var.name for var in variables if "_bin" in var.nam], "operation": "Add"})


# define the objective
#NOTE eq: min  sum_f(var_f * pricePerServing_f)
objective = tupObjective("minimize cost of nutritional diet", [], "min")
for food, pricing in food_info.items():
    effective_price = pricing["price"]/pricing["yield"]
    objective.expr.extend({
        "elements": [
            {"elements": [variables[food].name, effective_price],
                "operation": "Mul"}],
        "operation": "Add"
    })

# create an Optland model from the defined variables, constraints, and objective expression
model = OptlangHelper.define_model("minimize_nutrition_cost", variables, constraints, objective, True)

KeyError: 'Acorn squash'